In [1]:
import html
import re

def clean_file(path):
    f = open(path, 'r', encoding="utf-8")
    content = html.unescape(f.read())
    # print(content)
    search = re.search("<span><p class=............>(.*)</p></span>", content)
    # search = re.search("<span><p class=(.*)</p></span>", content)
    if search != None:
        body = re.sub("<.*?>", "", search.group(1))
        return body
    else:
        return ""

In [2]:
# development tests for clean_file
# should be able to extract body text, and any other features that might be nice.

path = ".\Adzhubei__Aleksei\March_12__1962\html\Alexei_Adzhubei-s_Account_of_His_Visit_to_Washington_to_the_Central_Committee_of_the_Communist_Party_of_the_Soviet_Union__0.html"
clean_file(path)


'TOP SECRETDuring my visit to Washington, Brazil, and passing through Mexico, I had a few meetings with US President John Kennedy, his brother, Robert Kennedy, and a few other figures from the President’s circle. In Brazil I met with President [João] Goulart, Prime Minister [Tancredo] Neves, Minister of International [Foreign] Affairs [San Tiago] Dantos, and a few other officials. In Mexico I had a short conversation with President [Adolfo] Lopez Mateos. Telegrams were dispatched to Moscow from these countries in which the character of these conversations was reported. I would like to report to the Central Committee a few more details and circumstances about the meetings and conversations which occurred in the USA which will help to complete the picture.Firstly, what surprised American journalists and journalists of other Western nations accredited to Washington was the heightened interest of American society in the very fact, as [columnist Walter] Lippmann expressed it, of the contact

In [3]:
# Obtaining metadata from folder structure.

from pathlib import Path
import os

problem_files = []
output = []
n = 0

for subdir, dirs, files in os.walk("."):
    for file in files:
        filepath = os.path.join(subdir, file)

        if filepath.endswith(".html"):
            n += 1
            patharr = filepath.split(os.sep)
            #print(filepath)
            #print('author_name: ' + patharr[1])
            #print('date: ' + patharr[2])
            #print('title: ' + patharr[4][:-5]) 
            
            current_file_info = [patharr[1], patharr[2], patharr[4][:-5], ""]

            try:
                current_file_info[3] = clean_file(filepath)
            except (FileNotFoundError):
                problem_files.append(("file not found", filepath))
            except (UnicodeDecodeError):
                problem_files.append(("unicode decode issues", filepath))
            
            output.append(current_file_info)

In [138]:
# Report on problem files
print("Total files processed: " + str(n))
print("Files with errors: " + str(len(problem_files)))

Total files processed: 778
Files with errors: 47


In [6]:
# Import cleaned data into dataframe

import pandas as pd

df = pd.DataFrame.from_records(output)
df

,0,1,2,3
0,Adzhubei__Aleksei,March_12__1962,Alexei_Adzhubei-s_Account_of_His_Visit_to_Wash...,"TOP SECRETDuring my visit to Washington, Brazi..."
1,Albania__Ministry_of_Foreign_Affairs,March_15__1971,Notes_on_a_Bulletin_of_the_Korean_News_Agency_0,[Handwritten document] Note Looking at the bul...
2,Alexandru__Boaba,April_06__1978,TELEGRAM_075_205_from_the_Romanian_Embassy_in_...,
3,Anda__Torleiv__1921-,October_21__1976,Telegram_from_the_Embassy_in_Beijing__-Smuggli...,ROYAL MINISTRY OF FOREIGN AFFAIRSCOPY NO:1: MI...
4,Anda__Torleiv__1921-,October_22__1976,Telegram_from_Norwegian_Ambassador_to_China_to...,"22.10.76, 09.34 amCOPY NO:1: MINISTER OF FOREI..."
...,...,...,...,...
773,Zhou__Enlai__1898-1976_,September_16__1952,Hand_delivered_note__Zhou_Enlai_to_Stalin__con...,"To Comrade STALIN, I.V.I send you a Russian tr..."
774,Zhou__Enlai__1898-1976_,September_20__1950,Telegram_from_Zhou_Enlai_to_Ni_Zhiliang_0,Comrade Ni Zhiliang:This is to acknowledge rec...
775,Zhou__Enlai__1898-1976_,September_29__1950,Telegram_from_Zhou_Enlai_to_Ni_Zhiliang_0,Ambassador Ni [Zhiliang]; also inform Chairman...
776,Zhou__Enlai__1898-1976_,September_30__1958,Memorandum_of_Conversation__Premier_Zhou_Recei...,Memorandum of Conversation: Premier Zhou Recei...


In [7]:
df.to_csv("data_clean")